# consumerdb

In [20]:
import pyspark 

In [1]:
df = sqlContext.read.format('orc').load("2022-09-19.orc")

In [2]:
df

DataFrame[s_no: int, n_slot_id: int, cust_id: int, in_source: string, coupon_code: string, credit: decimal(8,2), table_check: string, dt: string]

In [22]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- agent_id: integer (nullable = true)
 |-- n_slot_id: integer (nullable = true)
 |-- agent_total_order: integer (nullable = true)
 |-- seven_am_delivery_per: decimal(8,2) (nullable = true)
 |-- eight_am_delivery_per: decimal(8,2) (nullable = true)
 |-- nine_am_delivery_per: decimal(8,2) (nullable = true)
 |-- nine_thirty_am_delivery_per: decimal(8,2) (nullable = true)
 |-- ten_am_delivery_per: decimal(8,2) (nullable = true)
 |-- delivery_complaint_per: decimal(8,2) (nullable = true)
 |-- start_slot: integer (nullable = true)
 |-- end_slot: integer (nullable = true)
 |-- orders_delivered_by_seven_am: integer (nullable = true)
 |-- orders_delivered_by_eight_am: integer (nullable = true)
 |-- orders_delivered_by_nine_am: integer (nullable = true)
 |-- orders_delivered_by_nine_thirty_am: integer (nullable = true)
 |-- orders_delivered_by_ten_am: integer (nullable = true)
 |-- total_order_complaints: integer (nullable = true)
 |-- eight_am_disposit

In [4]:
df.show(n=1,truncate=False)

+---+--------+---------+-----------------+---------------------+---------------------+--------------------+---------------------------+-------------------+----------------------+----------+--------+----------------------------+----------------------------+---------------------------+----------------------------------+--------------------------+----------------------+------------------------+---------------------------------+--------------------------+
|id |agent_id|n_slot_id|agent_total_order|seven_am_delivery_per|eight_am_delivery_per|nine_am_delivery_per|nine_thirty_am_delivery_per|ten_am_delivery_per|delivery_complaint_per|start_slot|end_slot|orders_delivered_by_seven_am|orders_delivered_by_eight_am|orders_delivered_by_nine_am|orders_delivered_by_nine_thirty_am|orders_delivered_by_ten_am|total_order_complaints|eight_am_disposition_per|orders_delivered_by_six_thirty_am|six_thirty_am_delivery_per|
+---+--------+---------+-----------------+---------------------+---------------------+--

In [19]:
df = sqlContext.read.format('orc').load("9f7e79d0-db28-4bbd-9777-c0c29ea759bf.orc")
table_name = 'sub1'
print("CREATE TABLE IF NOT EXISTS hive.historical.full_temp_finance_"+str(table_name)+"(")
for i in df.columns:
    if((str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])=='String'):
        print(str(i)+"  varchar,")
    if((str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-9])=='Decimal'):
        print(str(i)+" "+str(df.schema[i].dataType).split("Type"))
    else:
        print(str(i)+" "+(str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])+",")
print("dt varchar")
print(")")
print("WITH( partitioned_by = array['dt'],")
print("external_location = 's3a://presto-prod-otipy/otipy-database/backup-full-load/temp_finance/"+str(table_name)+"',")
print("format='ORC');")
print("")
print("CALL system.sync_partition_metadata(schema_name=>'historical', table_name=>'"+str(table_name)+"', mode=>'FULL');")

CREATE TABLE IF NOT EXISTS hive.historical.full_temp_finance_sub1(
id Integer,
agent_id Integer,
n_slot_id Integer,
agent_total_order Integer,


TypeError: can only concatenate str (not "list") to str

In [32]:
call=["CALL system.sync_partition_metadata(schema_name=>'historical', table_name=>'full_temp_finance_sub1', mode=>'FULL')"]

In [34]:
import trino
from trino.dbapi import connect
conn = connect(host="presto.crofarm.com",port=443,catalog='hive',http_scheme='https',auth=trino.auth.BasicAuthentication("db.manager","crofarm"))
cur = conn.cursor()

create=["insert into hive.temp_finance.sub1 select *, cast('done' as varchar) as data_copied from hive.historical.full_temp_finance_sub1 where dt = (select max(dt) from hive.historical.full_temp_finance_sub1 where dt >= cast(current_date as varchar))"]
c=["CREATE TABLE IF NOT EXISTS hive.historical.full_temp_finance_sub1( id Integer, cat varchar, old_cat varchar, dt varchar) WITH( partitioned_by = array['dt'], external_location = 's3a://presto-prod-otipy/otipy-database/backup-full-load/temp_finance/sub1', format='ORC')"]
# m=["CREATE TABLE IF NOT EXISTS hive.historical.temp_finance_sub1( id Integer, cat varchar, old_cat varchar, dt varchar) WITH( partitioned_by = array['dt'], external_location = 's3a://presto-prod-otipy/otipy-database/backup-full-load/temp_finance/sub1', format='ORC')"]
# n=["Drop table historical.temp_finance_sub1"]
for i in create :  
    b = i
    print(b)
    cur.execute(b)
    print(cur.fetchall())

insert into hive.temp_finance.sub1 select *, cast('done' as varchar) as data_copied from hive.historical.full_temp_finance_sub1 where dt = (select max(dt) from hive.historical.full_temp_finance_sub1 where dt >= cast(current_date as varchar))
[[4919]]


In [ ]:
insert into hive.temp_finance.sub1 select *, cast('done' as varchar) as data_copied from hive.historical.temp_finance_sub1 where dt = (select max(dt) from hive.historical.temp_finance_sub1 where dt >= cast(current_date as varchar))

In [12]:
for i in df.columns:
    print(str(df.schema[i].dataType))

IntegerType
StringType
StringType
DecimalType(8,2)
IntegerType
IntegerType
BooleanType
LongType
LongType
IntegerType
IntegerType
IntegerType
IntegerType
StringType
IntegerType
IntegerType
BooleanType
IntegerType
IntegerType
StringType
IntegerType
IntegerType
IntegerType
DecimalType(6,2)
DecimalType(8,2)


# crofarmUsersdb

In [8]:
df = sqlContext.read.format('orc').load("2022-09-19.orc")
table_name = 'recharge_code'
create_table=[]
n=[]
for field in df.schema.fields:
    if(str(field.dataType).replace("Type","")=="String"):
        n.append(field.name +" "+str("varchar"))
    elif(str(field.dataType).replace("Type","")=="Long"):
        n.append(field.name +" "+str("bigint"))
    else:
        n.append(field.name +" "+str(field.dataType).replace("Type",""))
o=("CREATE TABLE IF NOT EXISTS hive.finance."+table_name+ '[%s]' % ', '.join(map(str, n)))
x = o.replace("[", "(")
x=x.replace("]",")")
x=(x+" WITH(partitioned_by = array['dt'], external_location = 's3a://presto-prod-otipy/otipy-database/finance/"+table_name+"', format='ORC')")
create_table.append(x)

In [ ]:
s3://presto-prod-otipy/otipy-database/finance/recharge_code/dt=2022-09-19/2022-09-19.orc

In [9]:
create_table

["CREATE TABLE IF NOT EXISTS hive.finance.recharge_code(s_no Integer, n_slot_id Integer, cust_id Integer, in_source varchar, coupon_code varchar, credit Decimal(8,2), table_check varchar, dt varchar) WITH(partitioned_by = array['dt'], external_location = 's3a://presto-prod-otipy/otipy-database/finance/recharge_code', format='ORC')"]

## temp_financedb

In [5]:
df = sqlContext.read.format('orc').load("2022-08-31.orc")
table_name = 'consumer_base'
print("CREATE TABLE IF NOT EXISTS hive.bizfin."+str(table_name)+"(")
for i in df.columns:
    
    if((str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])=='String'):
        print(str(i)+"  varchar,")
    elif((str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])=='Long'):
        print(str(i)+"  bigint,")
    else:
        print(str(i)+" "+(str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])+",")
print("dt varchar")
print(")")
print("WITH( partitioned_by = array['dt'],")
print("external_location = 's3a://presto-prod-otipy/full_load_1am/temp_finance/"+str(table_name)+"',")
print("format='ORC');")
print("")
print("CALL system.sync_partition_metadata(schema_name=>'bizfin', table_name=>'"+str(table_name)+"', mode=>'FULL');")

CREATE TABLE IF NOT EXISTS hive.bizfin.consumer_base(
yr  varchar,
mnth  varchar,
wk  varchar,
n_slot_id Integer,
dt  varchar,
polygon_id Integer,
polygon_name  varchar,
distributor_id Integer,
DC  varchar,
status  varchar,
prod_id Integer,
prod_name  varchar,
subcat  varchar,
new_subcat  varchar,
cat  varchar,
Prod_ord  bigint,
qty DecimalType(1,
SP_value DecimalType(1,
SP DecimalType(1,
WI_amt DecimalType(1,
ref_wd Double,
act_wd Double,
stf_qty DecimalType(1,
stf_value DecimalType(1,
mkt_cost DecimalType(1,
act_qty DecimalType(1,
SP_v2 DecimalType(1,
pp_wd_qty DecimalType(1,
total_orders  bigint,
PP_sys  bigint,
PP DecimalType(1,
PP_value DecimalType(1,
Margin DecimalType(1,
FNV_FMCG  varchar,
City_Name  varchar,
NCR_MMR  varchar,
date_partiton_key  varchar,
dt varchar
)
WITH( partitioned_by = array['dt'],
external_location = 's3a://presto-prod-otipy/full_load_1am/temp_finance/consumer_base',
format='ORC');

CALL system.sync_partition_metadata(schema_name=>'bizfin', table_name=>'con

## productsdb

In [9]:
df = sqlContext.read.format('orc').load("000da3db-3499-4f2c-af8a-b3edbbf69897.orc")
table_name = 'order_marketing_cost'
print("CREATE TABLE IF NOT EXISTS hive.products."+str(table_name)+"(")
for i in df.columns:
    if((str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])=='String'):
        print(str(i)+"  varchar,")
    else:
        print(str(i)+" "+(str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])+",")
print(")")
print("(")
print("external_location = 's3a://presto-prod-otipy/otipy-database/live/products/"+str(table_name)+"',")
print("format='ORC');")
print("")
print("CALL system.sync_partition_metadata(schema_name=>'', table_name=>'"+str(table_name)+"', mode=>'FULL');")

CREATE TABLE IF NOT EXISTS hive.products.order_marketing_cost(
id Integer,
order_id Integer,
prod_id Integer,
marketing_cost Integer,
entry_time Long,
updated_at Long,
purchase_discount Integer,
)
(
external_location = 's3a://presto-prod-otipy/otipy-database/live/products/order_marketing_cost',
format='ORC');

CALL system.sync_partition_metadata(schema_name=>'historical_data', table_name=>'order_marketing_cost', mode=>'FULL');


## partition 

In [2]:
    df = sqlContext.read.format('orc').load("2022-06-29.orc")
    table_name = 'products.distributor_product_mapping'
    print("CREATE TABLE IF NOT EXISTS hive.consumer."+str(table_name)+"(")
    for i in df.columns:
        if((str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])=='String'):
            print(str(i)+"  varchar,")
        else:
            print(str(i)+" "+(str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])+",")
    print("dt varchar")
    print(")")
    print("WITH( partitioned_by = array['dt'],")
    print("external_location = 's3a://presto-prod-otipy/partitioned_data/consumerdb/"+str(table_name)+"',")
    print("format='ORC');")
    print("")
    print("CALL system.sync_partition_metadata(schema_name=>'products', table_name=>'"+str(table_name)+"', mode=>'FULL');")

CREATE TABLE IF NOT EXISTS hive.consumer.products.distributor_product_mapping(
id Integer,
distributor_id Integer,
mandi_moq  varchar,
farm_moq  varchar,
mandi_purchase  varchar,
farm_purchase  varchar,
mandi_margin  varchar,
farm_margin  varchar,
is_active Boolean,
have_deal  varchar,
entry_time  varchar,
update_time  varchar,
product_id Integer,
is_standard Boolean,
farm_price_updated  varchar,
farm_price_validity  varchar,
mandi_price_updated  varchar,
mandi_price_validity  varchar,
purchase_moq  varchar,
specification  varchar,
old_price_updated  varchar,
old_purchase  varchar,
old_remark  varchar,
remark  varchar,
is_inv_linked Boolean,
master_product  varchar,
in_kg  varchar,
shelf_life Integer,
is_new Boolean,
op_cost  varchar,
op_cost_details  varchar,
dsm  varchar,
default_enabled Boolean,
pur_price  varchar,
igst  varchar,
stock_out_time Integer,
is_adv_inv_linked Boolean,
rejection_image Integer,
in_pcs Boolean,
deal_of_day Boolean,
rank Integer,
combo_prod_list  varchar,
is

In [28]:
df = sqlContext.read.format('orc').load("16ca457f-3206-4157-b9b5-94f0c8410c30.orc")
table_name = 'cr_address'
print("CREATE TABLE IF NOT EXISTS hive.crofarmUsers."+str(table_name)+"(")
for i in df.columns:
    if((str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])=='String'):
        print(str(i)+"  varchar,")
    else:
        print(str(i)+" "+(str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])+",")
print("dt varchar")
print(")")
print("WITH( partitioned_by = array['dt'],")
print("external_location = 's3a://presto-prod-otipy/partitioned_data/crofarmUsersdb/"+str(table_name)+"',")
print("format='ORC');")
print("")
print("CALL system.sync_partition_metadata(schema_name=>'crofarmusers', table_name=>'"+str(table_name)+"', mode=>'FULL');")

CREATE TABLE IF NOT EXISTS hive.crofarmUsers.cr_address(
id Integer,
line1  varchar,
line2  varchar,
latitude  varchar,
longitude  varchar,
pincode  varchar,
entry_time  varchar,
update_time  varchar,
city_id Integer,
place_id  varchar,
hindi_address  varchar,
area  varchar,
house_no  varchar,
locality  varchar,
tower_no  varchar,
grid_id  varchar,
bag_drop_lat  varchar,
bag_drop_long  varchar,
dt varchar
)
WITH( partitioned_by = array['dt'],
external_location = 's3a://presto-prod-otipy/partitioned_data/crofarmUsers/cr_address',
format='ORC');

CALL system.sync_partition_metadata(schema_name=>'crofarmUsers', table_name=>'cr_address', mode=>'FULL');


In [70]:
s='''`bvarchar
    locality	varchar
    tower_no	varchar
    grid_id	varchar
    bag_drop_lat	varchar
    bag_drop_long	varchar
    dt	varchar'''

In [ ]:
CREATE TABLE hive.historical_data.cr_address (
   id integer,
   line1 varchar,
   line2 varchar,
   latitude varchar,
   longitude varchar,
   pincode varchar,
   entry_time varchar,
   update_time varchar,
   city_id integer,
   place_id varchar,
   hindi_address varchar,
   area varchar,
   house_no varchar,
   locality varchar,
   tower_no varchar,
   grid_id varchar,
   bag_drop_lat varchar,
   bag_drop_long varchar,
   dt varchar
)
WITH (
   external_location = 's3a://presto-prod-otipy/historical_data/crofarmUsersdb/cr_address',
   format = 'ORC',
   partitioned_by = ARRAY['dt']
)

In [118]:
def remove(string):
    return "".join(string.split(" "))
data = a
df = pd.DataFrame([x.split('\t') for x in s.split('\n')],columns=['columns','dataType'])
table_name = 'cr_address'
print("CREATE TABLE IF NOT EXISTS hive.crofarmUsers."+str(table_name)+"(")
for i in range(len(df)):
    print(str(df['columns'][i])+" "+df['dataType'][i]+",")
print(")")
print("WITH( partitioned_by = array['dt'],")
print("external_location = 's3a://presto-prod-otipy/partitioned_data/crofarmUsersdb/"+str(table_name)+"',")
print("format='ORC');")
print("")
print("CALL system.sync_partition_metadata(schema_name=>'crofarmusers', table_name=>'"+str(table_name)+"', mode=>'FULL');")

CREATE TABLE IF NOT EXISTS hive.crofarmUsers.cr_address(
id integer,
    line1 varchar,
    line2 varchar,
    latitude varchar,
    longitude varchar,
    pincode varchar,
    entry_time varchar,
    update_time varchar,
    city_id integer,
    place_id varchar,
    hindi_address varchar,
    area varchar,
    house_no varchar,
    locality varchar,
    tower_no varchar,
    grid_id varchar,
    bag_drop_lat varchar,
    bag_drop_long varchar,
    dt varchar,
)
WITH( partitioned_by = array['dt'],
external_location = 's3a://presto-prod-otipy/partitioned_data/crofarmUsersdb/cr_address',
format='ORC');

CALL system.sync_partition_metadata(schema_name=>'crofarmusers', table_name=>'cr_address', mode=>'FULL');


In [83]:

table_name = 'cr_address'
print("CREATE TABLE IF NOT EXISTS hive.crofarmUsers."+str(table_name)+"(")
for i in df.:
    if((str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])=='String'):
        print(str(i)+"  varchar,")
    else:
        print(str(i)+" "+(str(df.schema[i].dataType)[:len(str(df.schema[i].dataType))-4])+",")
print("dt varchar")
print(")")
print("WITH( partitioned_by = array['dt'],")
print("external_location = 's3a://presto-prod-otipy/partitioned_data/crofarmUsersdb/"+str(table_name)+"',")
print("format='ORC');")
print("")
print("CALL system.sync_partition_metadata(schema_name=>'crofarmusers', table_name=>'"+str(table_name)+"', mode=>'FULL');")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (770983717.py, line 8)

,0
0,id\tinteger\nline1\tvarchar\nline2\tvarchar\nl...


In [88]:
df = sqlContext.read.format('orc').load("16ca457f-3206-4157-b9b5-94f0c8410c30.orc")

StructField(amount,StringType,true)
